In [1]:
from bs4 import BeautifulSoup
import requests

import multiprocessing as mp
import concurrent.futures

import time
import re
import os
import pandas as pd

In [2]:
def _parse_content(url):
    headers = {'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'}
    content = BeautifulSoup(requests.get(url, headers).content, 
                            'html.parser')
    return content

def _get_markets(inputParam):
    """helper function"""
    dir_output = r'C:\Users\ojell\Desktop\Oj\_Thesis\Data\news\businessworld'
    url = inputParam[0]
    category = inputParam[1]
    lstpage = inputParam[2]
    
    if lstpage == 9130:
        strt = 9001
    else:
        strt = lstpage - 299
    
    ArtUrl = []
    for pg in range(strt, lstpage + 1):
        content = _parse_content(url + f"page/{str(pg)}/")
        if pg % 50 == 0: time.sleep(10)
        next10 = content.find('div', {'class': 'td-ss-main-content'}).find_all('div', {'class': 'td-module-thumb'})
        ArtUrl = ArtUrl + [item.find('a').get('href') for item in next10]
        dct = {"Article Url": ArtUrl, "Category": category}
    df = pd.DataFrame.from_dict(dct)
    filename = category.lower() + f'_{lstpage}' + '.csv'
    df.to_csv(os.path.join(dir_output, filename), index=False)
    return df

def _get_markets_d(inputParam):
    pass
    
    
def par_main_Markets(news_url):    
    content = _parse_content(news_url)

    MarketContent = content.find('div', {'class': 'td-ss-main-content'}).find_all('div')
    
    pagetab = content.find('div', {'class': 'page-nav td-pb-padding-side'}).find('span', {'class': 'pages'}).text
    lstpg = int(re.sub(r'[^\w\s]', '', pagetab[-(len(pagetab) - 10):]))
    #lstpg_li = [i for i in range(1, lstpg + 1) if i % 300 == 0 or i == lstpg]
    lstpg_li = [1500, 3300, 4500, 4800, 6600, 9130]
        
    output = 0
    with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as pool:
        futures = [pool.submit(_get_markets, 
                               ['https://www.bworldonline.com/markets/', 'Markets', lstpg_li[i]]) for i in range(0, len(lstpg_li))]
        for f in concurrent.futures.as_completed(futures):
            output += 1
    return


newsMarkets = 'https://www.bworldonline.com/markets/'

In [3]:
start = time.time()
markets = par_main_Markets(newsMarkets)
par_time = time.time() - start
print(markets)
#print(f"Parallel Output: {len(test)}")
print(f"Parallel elapsed time: {round(par_time/60, 2)} mins")

None
Parallel elapsed time: 8.07 mins


### 